# MeanHamilMinimizer_rigetti_scipy
* Feedback loop between Qubiter and Rigetti QVM
* minimization via scipy.optimize.minimize

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/jupyter/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/jupyter/Notebooks/Quantum/qubiter


In [2]:
from adv_applications.MeanHamilMinimizer_rigetti import *

loaded OneBitGates, WITHOUT autograd.numpy


In [3]:
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from pyquil import get_qc

In [4]:
qvm_url = "http://localhost:5000"
compiler_server_address = "tcp://localhost:5555"
forest_url = "https://forest-server.qcs.rigetti.com"
qvm = QVMConnection(endpoint=qvm_url, compiler_endpoint=compiler_server_address)

## First Example (taken from Pennylane docs). 

In [5]:
qc = get_qc('2q-qvm')

In [6]:
num_bits = 2
file_prefix = 'io_folder/mean_hamil_rigetti_test1'
emb = CktEmbedder(num_bits, num_bits)
wr = SEO_writer(file_prefix, emb)
wr.write_Rx(0, rads='#1')
wr.write_Ry(0, rads='#2')
wr.close_files()

In [7]:
wr.print_eng_file()

ROTX	#1	AT	0
ROTY	#2	AT	0



In [8]:
wr.print_pic_file()

|   Rx  
|   Ry  



In [9]:
init_var_num_to_rads = {1: 0., 2: 0.}

In [10]:
fun_name_to_fun = None

In [11]:
hamil = QubitOperator('Z0', 1.)
print('hamil=\n', hamil)

hamil=
 1.0 [Z0]


In [12]:
num_samples = 100
rand_seed = 1234
print_hiatus = 10
verbose = False

In [13]:
mini = MeanHamilMinimizer_rigetti(qc, file_prefix, num_bits, hamil,
                init_var_num_to_rads, fun_name_to_fun,
                num_samples=num_samples, rand_seed=rand_seed,
                print_hiatus=print_hiatus, verbose=verbose)

In [14]:
mini.translator.print_aqasm_file()


pg += RX(rads1*(-2), 0)
pg += RY(rads2*(-2), 0)




In [15]:
print(mini.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[2]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RX(rads1*-2) 0
RY(rads2*-2) 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [16]:
mini.find_min(interface='scipy', method='Powell')

x_val~ (#1, #2)
iter=0, cost=1.000000, x_val=0.000000, 0.000000
iter=10, cost=-0.980000, x_val=1.470248, 0.000000
iter=20, cost=-0.980000, x_val=1.500930, -0.005474
iter=30, cost=-1.000000, x_val=1.500930, -0.004741
iter=40, cost=-1.000000, x_val=1.516570, -0.004790
iter=50, cost=-1.000000, x_val=1.505621, -0.004756
iter=60, cost=-0.960000, x_val=1.505621, -0.052075
iter=70, cost=-1.000000, x_val=1.505621, -0.005034
iter=80, cost=-1.000000, x_val=1.505621, -0.004774
iter=90, cost=-0.980000, x_val=1.507413, -0.004780
iter=100, cost=-1.000000, x_val=1.506598, -0.004778


   direc: array([[ 0.00000000e+00,  1.00000000e+00],
       [ 4.69075598e-03, -1.48170301e-05]])
     fun: -1.0
 message: 'Optimization terminated successfully.'
    nfev: 102
     nit: 2
  status: 0
 success: True
       x: array([ 1.5065977 , -0.00477785])

## Second, more complicated example

In [17]:
qc = get_qc('4q-qvm')

In [18]:
num_bits = 4
file_prefix = 'io_folder/mean_hamil_rigetti_test2'
emb = CktEmbedder(num_bits, num_bits)
wr = SEO_writer(file_prefix, emb)
wr.write_Ry(2, rads=np.pi/7)
wr.write_Rx(1, rads='#2*.5')
# wr.write_Rn(3, rads_list=['#1', '-#1*3', '#2'])
wr.write_Rx(1, rads='-#1*.3')
wr.write_cnot(2, 3)
wr.write_bit_swap(1, 2)
wr.close_files()

In [19]:
wr.print_eng_file()

ROTY	25.714285714285715	AT	2
ROTX	#2*.5	AT	1
ROTX	-#1*.3	AT	1
SIGX	AT	3	IF	2T
SWAP	2	1



In [20]:
wr.print_pic_file()

|   Ry  |   |   
|   |   Rx  |   
|   |   Rx  |   
X---@   |   |   
|   <--->   |   



In [21]:
fun_name_to_fun = None

In [22]:
init_var_num_to_rads = {1: 2.1, 2: 3.4}

In [23]:
hamil = QubitOperator('X1 Y3 X1 Y1', .4) + QubitOperator('Y2 X1', .7)
print('hamil=\n', hamil)

hamil=
 0.7 [X1 Y2] +
0.4 [Y1 Y3]


In [24]:
num_samples = 100
rand_seed = 1234
print_hiatus = 25
verbose = False

In [25]:
mini = MeanHamilMinimizer_rigetti(qc, file_prefix, num_bits, hamil,
                init_var_num_to_rads, fun_name_to_fun,
                num_samples=num_samples, rand_seed=rand_seed,
                print_hiatus=print_hiatus, verbose=verbose)

In [26]:
mini.translator.print_aqasm_file()


pg += RY(-0.8975979010256552, 2)
pg += RX(rads2*.5*(-2), 1)
pg += RX(-rads1*.3*(-2), 1)
pg += CNOT(2, 3)
pg += SWAP(2, 1)




In [27]:
print(mini.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[4]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RY(-2*pi/7) 2
RX(rads2*0.5*-2) 1
RX(-1*rads1*0.3*-2) 1
CNOT 2 3
SWAP 2 1
RY(-pi/2) 1
RX(pi/2) 2
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]
MEASURE 3 ro[3]



In [28]:
mini.find_min(interface='scipy', method='Powell')

x_val~ (#1, #2)
iter=0, cost=0.488000, x_val=2.100000, 3.400000
iter=25, cost=-0.444000, x_val=8.493284, 3.786991
iter=50, cost=-0.560000, x_val=9.111318, 3.726238
iter=75, cost=-0.522000, x_val=9.095733, 3.670510


   direc: array([[1., 0.],
       [0., 1.]])
     fun: -0.6299999999999999
 message: 'Optimization terminated successfully.'
    nfev: 95
     nit: 3
  status: 0
 success: True
       x: array([9.09593519, 3.77643135])